# ExperimentalDesign

In this notebook we will be using a random-search algorithm to choose the optimal design for our experiment. Creating random designs (without evaluation) in fMRI world often leads to under powered designs because the randomisation does not consider the lagged hemodynamic response and how one trial's response may bleed into another trial's response. This is more of a concern in event related designs, but still something to consider in block designs as well.

The video below from Jeanette Mumford helps illuminate task design and to consider what we want from our design

### Learning Objectives
- Describe the difference between estimation and detection
- Understand why choosing a random design can lead to reduced power
- State the rule of thumb for a good variance inflation factor (or variance reduction factor)
- Learn the options of optseq2 (freesurfer tool) and how to use the tool to design an efficient experimental design
- Make an experimental design for your group project (or for a project you may do in the future)

In [ ]:
%%bash
# Pre lab setup
res=$(which optseq2)

if [ "${res}" == "" ]; then
    sed -ie 's|export PATH=/home/brain/.packages/miniconda3/bin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games|export PATH=/home/brain/.packages/miniconda3/bin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:$PATH|g' ${HOME}/.bashrc
    
    source ~/.bashrc

    rm ${HOME}/.bashrce
fi



# now close this notebook and open it again

In [ ]:
# import necessary packages
%matplotlib inline
import pandas as pd

In [ ]:
from IPython.display import HTML

# If you don't hear this video and you're viewing from virtual machine, you may need to 
# set up audio in virtual machine settings (you'll need to do this when the machine is powered off)
# Jeanette Mumford video
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/FD4ztsoYvSY" frameborder="0" gesture="media" allow="encrypted-media" allowfullscreen></iframe>')



## Building An Experiment
In lab today we will be considering the flanker experiment we covered in the previous lab (07-Lab).

![flanker](img/task.png)

The relevant parameters are:
- the number of event types (3)
- the duration of the event (200 ms)
- the duration of the response period (1800 ms) (immediately after the event)
- the number of presentations for each event type (40)
- the length of the scan (315 volumes = 630 seconds)
- the contrast(s) of interest (incongruent - congruent).

In [ ]:
%%bash
# display the help of optseq for us to see all the parameters
optseq2 --help

## Relevant optseq2 Parameters
#### --ntp 315
 - DOC: Number of time points to be acquired during the scan.  This should be
for one 'run' not for the entire session. The Total Scanning Time
is the number of time points times the TR plus the prescan period,
ie, tScanTot = Ntp*TR+tPreScan. 
- ANS: The flanker scan has 315 volumes.

#### --tr 2
 - DOC: Time between functional volumes (in seconds).
 - ANS: For this scan the TR is 2 seconds
 
#### --psdwin 0 20 0.2
 - DOC: Specifications for the FIR event response window. It will be assumed that 
the entire response can be captured within this window. PSDMin is the 
minimum PostStimulus Delay (PSD), PSDMax is the maximum PSD. dPSD 
is the sampling interval within the window. dPSD is optional; if 
left unset, it will default to the TR. dPSD controls how finely spaced  
the event onsets can be scheduled (ie, the onsets will only appear at  
integer multiples of the dPSD). 
 -  ANS: I've set the relevant hemodynamic response period to be between 0 and 20 seconds.
    This means an event at 8 seconds will have an influence
    on the design matrix from 8 - 28 seconds. Real hemodynamic responses can have a residual
    effect up to 36 seconds after the event, but the majority of the effect *should* be
    done after 20 seconds. The third parameter is the resolution of the design matrix.
    This has to be as small as the event duration or a factor smaller (e.g. 0.1, 0.05, etc).

#### --ev con 0.2 40 --ev inc 0.2 40 --ev neu 0.2 40
- DOC: Event Type specification. The label is just a text label (which may be 
more informative than a numeric id). Duration is the number of seconds 
that the stimulus will be presented; it should be an integer multiple 
of the dPSD (see --psdwin). Nrepetitions is the number of times that 
this event type will be presented during the course of the run. The 
number of repetitions can be optimized using the --repvar option. Use 
a different --ev flag for each event type. NOTE: DO NOT INCLUDE THE 
NULL STIMULUS AS AN EVENT TYPE.  The total stimulation time, tStimTot, 
equals the product of the duration and the number of repetitions 
summed over all the events. It should be obvious that the total 
stimulation time must be less than the total scanning time. 
- ANS: The flanker task has three event types: congruent (con), 
  incongruent (inc), and neutral (neu), each of the stimuli
  will be presented for 0.2 seconds (200 milli-seconds) and repeated
  40 times each.

#### --tnullmin 3.8
- DOC: Force the NULL stimulus to be at least tNullMin sec between stimuli.
Note that this means that the stimulus duration + tNullMin must be
an integer multiple of the dPSD.
- ANS: This enforces participants will have the 1.8 second period to respond
  after the 0.2 second event, meaning at least 2 seconds of "rest" will be given
  to participants after each trial.

#### --tnullmax 20
- DOC: Limit the maximum duration of the NULL stimulus to be tNullMax sec.
 Note: it may not be possible for a given parameter set to keep the NULL 
stimulus below a certain amount. In this case, the following error 
message will be printed out 'ERROR: could not enforce tNullMax'. By
default, tNullMax is infinite. 
- ANS: A pratical limitation to constrain the creation of designs
  with loooong rest periods between trials.

#### --nsearch 1000
- DOC: Search over Nsearch iterations. optseq will randomly construct Nsearch 
schedules, compute the cost of each one, and keep the ones with the 
highest cost. It is not permitted to specify both Nsearch and Tsearch. 
- ANS: I'm generating 1,000 designs for time's sake, you will want 
  a much larger number (i.e. 10,000 or even 100,000) for actual design generation
  
#### --evc -1 1 0
- DOC: Optimize based on a contrast of the event types. Ci is the contrast 
weight for event type i. There must be as many weights as event types. 
Weights are NOT renormalized such that they sum to 1. 
- ANS: each number corresponds to each `--ev` I've specified in the order I specified them
  In this case the -1 corresponds to the congruent condition, 1 corresponds to the incongruent condition
  and 0 refers to the neutral condition. The contrast in words means (incongruent - congruent).

#### --seed 123
- DOC: Initialize the random number generator to seedval. If no seedval is 
specified, then one will be picked based on the time of day. optseq2 
uses drand48(). 
- ANS: To make sure my analysis is reproducible

#### --nkeep 5
- DOC: Save nKeep of the best schedules. Increasing this number does not 
substantially increase the search time, so it is a good idea to  
specify more than you think you will need. 
- ANS: keeping the top five (in case I want to randomise which version I use across participants)

#### --o flanker_stims
- DOC:Save schedules in outstem-RRR.par, where RRR is the 3-digit 
zero-padded schedule rank number (there will be nKeep of them). 
The schedules will be saved in the Paradigm File Format (see below).
- ANS: A useful output name

In [ ]:
%%bash
optseq2 \
--tprescan -8 `# number of seconds to wait before showing stimuli (allows scanner to reach steady state)` \
--ntp 315 `# number of volumes in the experiment` \
--tr 2 `# repetition time (how long it takes to scan the entire brain once)` \
--psdwin 0 20 0.2 `# The first two numbers give the time range of the HRF, and the third is resolution of the design` \
--ev con 0.2 40 `# congruent (con: 0.2 second duration, 40 repetitions)` \
--ev inc 0.2 40 `# incongruent (inc: 0.2 second duration, 40 repetitions)` \
--ev neu 0.2 40 `# neutral (neu: 0.2 second duration, 40 repetitions)` \
--tnullmin 3.8 `# The minimum time between events. 3.8 - 1.8 (response period) = 2.0 seconds (the real amount of non-processing time)` \
--tnullmax 20 `# The maximum time between events` \
--nsearch 1000 `# The number of designs to consider` \
--evc -1 1 0 `# The contrast of interest for the experiment (inc - con)` \
--seed 123 `# set random seed to get consistent results` \
--nkeep 5 `# keep the top five designs` \
--o flanker_events_orig `# the output base name` &> /dev/null

In [ ]:


# open and read the summary file
def parse_optseq_summary(summary_file):
    with open(summary_file) as sf:
        n = -1
        # output collector in a list
        outer_lst = []
        # for each line
        for line in sf:
            # We are interested the line starting Rank and the five lines after that
            # representing the five designs we made
            if line.startswith("Rank"):
                n = 5
            # if the previous statement set n to 5 store the current and the next 
            # five lines into a list.
            if n >= 0:
                # remove the newline character, split by spaces, and remove null string elements.
                outer_lst.append([x for x in line.replace('\n', '').split(' ') if x is not ''])
                n -= 1
    return outer_lst

# the summary file that contains useful information about the designs
sum_file = 'flanker_events_orig.sum'
# run the function
optseq_info = parse_optseq_summary(sum_file)
# the top list contains the headers for the dataframe
headers = optseq_info.pop(0)

# set the dataframe
df_orig = pd.DataFrame(optseq_info, columns=headers)
# make the contents numeric (were strings)
df_orig = df_orig.apply(lambda x: pd.to_numeric(x))
# display the first five elements (which are coincidently all the elements)
display(df_orig.head())
display(df_orig.describe())
# plot the Variance Reduction Factor
df_orig['VRFAvg'].plot.bar(title='Variance Reduction Factor Average', x='designs')


### Variance Reduction Factors (VRF)
This is a measure of how correlated the regressors (e.g. con, inc, and neu) are with each other. For example, if a congruent trial always followed 2 seconds after an incongruent trial the regressors would be highly correlated resulting in a very low variance reduction factor. Each condition (e.g. congruent, incongruent, and neutral) will have their own variance reduction factor. If $i$ stands for a particular condition $i$ (which can take the value of congruent, incongruent, or neutral in this instance), and $n$ represents the number of conditions (in this case 3), then we can calculate the average variance reduction factor like so:

$$1/n\int\limits_{i=1}^n 1 - R_i^2$$

Where $R_i^2$ is the r-squared component for that particular condition. In other words, $R_i^2$ measures the amount of variance in condition $i$ that can be explained by the other conditions. To give some intuition, a small variance reduction factor means the conditions are highly correlated with each other $R^2 = 0.99; 1 - R^2 = 0.01$, whereas a small variance reduction factor means the conditions are largely independent $R^2 = 0.01; 1 - R^2 = 0.99$. Unfortunately, outside of optseq, people often use variance inflation factor (VIF) when describing colinearity. Thankfully, the average variance inflation factor equation is just the inverse of the variance reduction factor equation.

$$1/n\int\limits_{i=1}^n 1/(1 - R_i^2)$$

Jeanette Mumford gives the rule of thumb the average VIF should ideally be under 5 or just under 10 if there are certain design constraints. So writing those in terms of variance reduction factors, they should be greater than 0.2, or at least greater than 0.1 in a pinch.

### VIF RULE OF THUMB
- VIF < 5 (or 10)

### VRF RULE OF THUMB
- VRF > 0.2 (or 0.1)


### Optseq2 Resources
- [optseq2 website](http://surfer.nmr.mgh.harvard.edu/optseq/)
- [Andrew Jahn Tutorial](https://www.youtube.com/watch?v=MIx_PN4FkKk)


### Other Resources
- [Excellent Practical Guide to Study Design](http://imaging.mrc-cbu.cam.ac.uk/imaging/DesignEfficiency)

## Practice Problems

1a) When we set `psdwin 0 20 0.2`, we essentially told optseq2 to smear the event to have an impact on the design for 20 seconds (as opposed to 0.2 seconds). What do you think would happen if we instead set `psdwin 0 4 0.2`. How would you predict this would impact the VRF (or inversely the VIF)? 

:Prediction Here:

In [ ]:
# Code Here

1b) Compare the VRFs you got from 1a with the VRFs calculated from class, you can graph them both or use the describe method on the dataframe (e.g. `df_orig.describe()`)

In [ ]:
# Code Here

:Describe Results:

2) With your group, think up of a experiment and come up with the optseq2 command to optimize your experimental design (remember to set `--nsearch 10000`)

In [ ]:
# Code Here